In [1]:
# script made on 16gb of ram
import os
import io
import os.path
import random
import pickle
import zipfile
import nltk
from math import ceil
from copy import deepcopy
import numpy as np
from tqdm import tqdm
import pandas as pd
import tensorflow as tf      #the progress bar
import en_core_web_sm as en  #from the spaCy library, https://spacy.io/usage/
from sklearn.metrics import f1_score

from  tensorflow.keras.backend import binary_crossentropy
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers


Using TensorFlow backend.


In [2]:
filename = "./data/train.csv"
testname = "./data/test.csv"
output = "./data/output/"

In [3]:
# enb_path = 'input/embeddings/wiki-news-300d-1M/' + 'wiki-news-300d-1M.vec'
# enb_path = 'input/embeddings/GoogleNews-vectors-negative300/'
# enb_path = 'input/embeddings/paragram_300_sl999/'
enb_path = "./data/embeddings/glove.840B.300d/glove.840B.300d.txt"

In [4]:
class Batch():
    def __init__(self, data, labels, size = 64):
        self.data = data
        self.lenght = len(self.data )
        del data
        self.labels = labels
        self.size = size
        self.index = 0
    
    def __next__(self):
        
        
        if (self.index >= self.lenght):
            raise StopIteration()  
        
        if (self.index + self.size > self.lenght):
            self.size = self.lenght - self.index 
        
        self.index += self.size  
        
        return self.data[self.index-self.size:self.index],self.labels[self.index-self.size:self.index]        
    
    def __len__(self):
        return ceil(len(self.data)/self.size)
    
    def reset(self):
        self.index = 0
    
    def __iter__(self):
        return self

In [5]:
#global variables to save memory
# _t is for tokenized
embeddings = []      #np array of embeddings
vocab = set([])          #the vocabulary from the enbeding file
questions_train = []    #every question tokenized in an array
questions_test = []
#text_t = []         #all the questions concatenated and tokenized
word_to_index = {}
index_to_word = {}

In [6]:
def tokenize_questions(text,nr_to_delete = 0):
    #nlp = en.load(tagger=False, entity=False)      #load the tokenizer
    #global text_t       #every token in the original order
    
    #text_t = [] 
    questions_tokenized = []
    
    bar  = tqdm(total = len(text))
    
    for batch in text:  #tqdm is the progress bar
        tokens = nltk.word_tokenize(batch)
        batch_tokenized = []
        for token in tokens:
            batch_tokenized.append(token)
        questions_tokenized.append(batch_tokenized)
        bar.update(1)
    bar.close()
    return questions_tokenized

In [7]:
train_df = pd.read_csv(filename)    #test_t is for now a dataFrame

In [8]:
test_df = pd.read_csv(testname)

In [9]:
hashes = test_df.values[:,1].tolist()

In [10]:
y = train_df.values[:,2].tolist()

In [11]:
# loadig the qustions tokenized, and all qustions concatenated tokenized

if not os.path.isfile(output + "questions_test"):
    questions_test = tokenize_questions( test_df.values[:,1].tolist() )   
    with  open(output + "questions_test",'wb')  as file:
        pickle.dump(questions_test, file)
    if not os.path.isfile(output + "questions_train"):
        questions_train = tokenize_questions( train_df.values[:,1].tolist() )
        with  open(output + "questions_train",'wb')  as file:
            pickle.dump(questions_train, file)
            
    #pickle.dump(text_t, open(output + "text_t",'wb') )
else:
    print("Loading data...")
    with open(output + "questions_train", "rb" ) as file:
        questions_train = pickle.load( file )
    with open(output + "questions_test", "rb" ) as file:
        questions_test = pickle.load( file )
        
    #text_t = pickle.load( open(output + "text_t", "rb" ) )
    print("Nr of questions train: ",len(questions_train))
    print("Nr of questions test : ",len(questions_test))
    #print("Nr of tokens:    ",len(text_t))

print('The fist question tokenized from train: ', questions_train[0][:8])
print('The fist question tokenized from test : ', questions_test[0][:10])

Loading data...
Nr of questions train:  1306122
Nr of questions test :  56370
The fist question tokenized from train:  ['How', 'did', 'Quebec', 'nationalists', 'see', 'their', 'province', 'as']
The fist question tokenized from test :  ['My', 'voice', 'range', 'is', 'A2-C5', '.', 'My', 'chest', 'voice', 'goes']


In [12]:
# saving the words from all the questions so we can remove 
# the ones that does not appear from embedings to save memory

def get_questions_vocab(*args):     
    vocab = set([])
    for questions_set in args:
        for question in questions_set:
            for word in question:
                vocab.add(word)
    vocab.add('UNK') #we add an word for the unknown words in the future unseen questions 
    return vocab
    
vocab = get_questions_vocab(questions_train,questions_test)
print("Nr of unique word in questions",len(vocab))

Nr of unique word in questions 314974


In [13]:
def load_embeddings(enb_path,vocab):    
    
    print("Loading embeddings...")
    
    with open(enb_path, 'r', encoding ='utf-8') as file:
    
        num_lines = sum(1 for line in file)
        bar  = tqdm(total = num_lines)
    
        file.seek(0)
    
        vocab_enb = []
        embeddings = []      
        for line in file:
            bar.update(1)
            array = line[:-1].split(' ')
            word = str(array[0])
            if word in vocab:
                vocab_enb.append(word)
                embeddings.append(np.array(array[1:]).astype('float32'))      
        bar.close()
        
    embeddings = np.stack(embeddings)
    
    return vocab_enb, embeddings

In [14]:
if not os.path.isfile(output + "embeddings"):
    vocab, embeddings = load_embeddings(enb_path,vocab)
    with open( output + "embeddings",'wb') as file:
        pickle.dump(embeddings, file)
    with open( output + "vocabulary",'wb') as file:
        pickle.dump(vocab,open( output + "vocabulary",'wb'))
else:
    print("Loading embeddings...")   
    with open(output + "embeddings", "rb" ) as file:
        embeddings = pickle.load( file )
    with open( output + "vocabulary", "rb" ) as file:
        vocab = pickle.load( file )
print('Embedings size:  ',embeddings.shape)
print('Vocabulary size: ', len(vocab))

print('\nThe word "'+ vocab[2]+'" with his enmeding: \n' , embeddings[2][:5], "... ", embeddings[2][-5:] )

Loading embeddings...
Embedings size:   (200543, 300)
Vocabulary size:  200543

The word "the" with his enmeding: 
 [ 0.27204  -0.06203  -0.1884    0.023225 -0.018158] ...  [-0.018168  0.11407   0.13015  -0.18317   0.1323  ]


In [15]:
def initialze_padding(vocab,embeddings):
    #we need to add the padding word to our vocabulary
    #we need to add the padding word to our embedding matrix
    
    print("Adding the padding char to vocab and embeddings...")
    vocab = ['/pad']+vocab
    pad_emb = []
    for i in range(len(embeddings[0])):
        pad_emb.append(0)
    pad_emb = np.array(pad_emb)
    embeddings = np.vstack([pad_emb,embeddings])
    
    return vocab, embeddings

def create_dictionaries(vocab):
    word_to_index = {}
    index_to_word = {}
    
    for idx, word in enumerate(vocab): #create the two dictionaries
        word_to_index[word] = idx
        index_to_word[idx] = word
        
    return word_to_index, index_to_word

def index_question_to_words(question_indexes):
    question_words = ""
    for word in question_indexes:
        question_words += index_to_word[word] + ' '
    return question_words

def token_to_index(questions_tokenized, vocab):
    
    for question in questions_tokenized: #chage the input from strings to ids
        for idx,word_token in enumerate(question): #replace the text with the word indexes
             question[idx] = word_to_index.get(word_token, 89770) #if the token is not in vocab put "UNK"
    return questions_tokenized

def pad_questions(data, max_len = 60):
    
    print("Padding questions...")
    data_aux = deepcopy(data)
    
    padded_count = 0
    cut_count = 0
    for idx in tqdm(range(len(data_aux))):
        question_len = len(data_aux[idx])
        if question_len <= max_len:
            padded_count += 1
            for i in range(question_len, max_len):
                data_aux[idx].append(0)
        elif question_len > max_len:
            cut_count += 1
            data_aux[idx] = data_aux[idx][:max_len]
        
        if len(data_aux[idx])>max_len:
            print(question_len)
            print(question)
            break
            
    print("Padded :", padded_count)
    print("Cut    :", cut_count)
    return data_aux

In [16]:
vocab,embeddings = initialze_padding(vocab,embeddings)

word_to_index, index_to_word = create_dictionaries(vocab)

Adding the padding char to vocab and embeddings...


In [17]:
print('The word UNK to index       :', word_to_index['UNK'])
print('The intex for UNK to string :', index_to_word[89770])
print('The shape of enbedings      :', embeddings.shape)
print('The first 5words from vocab :',vocab[:10])

The word UNK to index       : 89770
The intex for UNK to string : UNK
The shape of enbedings      : (200544, 300)
The first 5words from vocab : ['/pad', ',', '.', 'the', 'and', 'to', 'of', 'a', 'in', ':']


In [18]:
questions_train = token_to_index(questions_train,vocab)
questions_test = token_to_index(questions_test,vocab)

questions_train = pad_questions(questions_train)
questions_test = pad_questions(questions_test)

Padding questions...


100%|███████████████████████████████████████████████████████████████| 1306122/1306122 [00:05<00:00, 250989.06it/s]


Padded : 1305836
Cut    : 286
Padding questions...


100%|███████████████████████████████████████████████████████████████████| 56370/56370 [00:00<00:00, 233599.82it/s]


Padded : 56356
Cut    : 14


In [19]:
print("First train question to intdexes :")
print(index_question_to_words(questions_train[0][:17]))
print(questions_train[0][:18])

print("\nFirst test question to intdexes  :")
print(index_question_to_words(questions_test[0][:17])+"...")
print(questions_test[0][:18])

First train question to intdexes :
How did Quebec nationalists see their province as a nation in the 1960s ? /pad /pad /pad 
[254, 127, 9707, 53520, 118, 58, 7523, 28, 7, 1979, 8, 3, 10166, 37, 0, 0, 0, 0]

First test question to intdexes  :
My voice range is UNK . My chest voice goes up to F4 . Included sample in ...
[208, 1282, 679, 10, 89770, 2, 208, 4411, 1282, 855, 61, 5, 40223, 2, 8870, 2617, 8, 44]


In [20]:
def embedding_lookup(batch):
    global embeddings
    
    batch_embedding = []
    
    for int_question in batch:
        question_embedding = []

        for int_word in int_question:
            try:
                question_embedding.append(embeddings[int_word])
            except Exception:
                print(int_word)
        
        enb = np.stack(question_embedding)
        batch_embedding.append(enb)
    for el in batch_embedding:
        if len(el)!=60:
            print(el.shape)
    return np.stack(batch_embedding)

In [1]:
def confusion_matrix(label, prediction):
    true_positives = 0
    false_positives = 0
    true_negatives = 0
    false_negatives = 0
 
    for i in range(0, len(label)):
        if prediction[i] == 1:
            if prediction[i] == label[i]:
                true_positives += 1
            else:
                false_positives += 1
        else:
            if prediction[i] == label[i]:
                true_negatives += 1
            else:
                false_negatives += 1
    return  true_positives, false_positives, true_negatives, false_negatives

def scores(thresholds,output,label):
    best_f1 = 0.0
    best_out = []
    thresh = 0.0
    for th_val in thresholds:
                aux_output = np.copy(output)
                aux_output[aux_output<=th_val] = bool(0)
                aux_output[aux_output>th_val] = bool(1)

                true_positives, false_positives, true_negatives, false_negatives = confusion_matrix(label,aux_output)
                accuracy = (true_positives + true_negatives) / (true_positives \
                            + true_negatives + false_positives + false_negatives)
                
                if true_positives + false_positives == 0:
                    precision = 0.001
                else:
                    precision  = true_positives / (true_positives + false_positives)
                
                if true_positives == 0:
                    recall = 0.001
                else:
                    recall = true_positives / (true_positives + false_negatives)
                
                if precision == 0 or recall == 0:
                    f1_score = 0.001
                else:
                    f1_score = 2 / \
                        ((1 / precision)\
                         + (1 / recall))

                print('   Thresholds {:.2f}:: Acc: {:.4f} | Precision: {:.4f} | Recall: {:.4f} | F1: {:.4f}'.format(
                th_val, accuracy, precision, recall, f1_score))

                if f1_score > best_f1:
                    best_out = aux_output
                    best_f1 = f1_score
                    thresh = th_val
    return thresh,best_f1,best_out

def evaluate(session,questions,batch_size,predictions,x):
    #Saving the predictions on the competition train for the our best scor so far
    print("Predicting for test...")
    output = []
    batches = Batch(questions_test, np.zeros(len(questions_test),), batch_size)
    for batch in tqdm(batches):
        batch, label =  batch
        batch = embedding_lookup( batch )
        label = np.array(label)
        batch_output = session.run(predictions, {x: batch})
        output = np.concatenate((output, batch_output), axis=0)
    return output

class Stats:
    def __init__(self,test_data,session,thresholds,batch_size,predictions,x):
        self.f1_train, self.f1_val = [0, 0], [0, 0]
        self.best_result = {'output':[],'f1_score':0,'threshold':0, 'epoch':0, 'epoch_selection_score' :0}
        self.test_data = test_data
        self.output = []
        self.session = session
        self.thresholds = thresholds
        self.batch_size = batch_size
        self.predictions = predictions
        self.x = x

    def check(self,epoch,loss):
        epoch_selection_score = self.f1_val[1]
        th  = self.f1_val[0]
        f1 = self.f1_val[1]
        print("             ___________________________________")
        print("            |Epoch nr. {} recap ({:.4f}):        |". format(epoch,loss))
        print("            |Train: threshold: {:.2f} F1: {:.4f}  |".format(self.f1_train[0],self.f1_train[1]))
        print("            |Val  : threshold: {:.2f} F1: {:.4f}  |".format(self.f1_val[0],self.f1_val[1]))
        print("            |___________________________________|")

        if epoch_selection_score > self.best_result['epoch_selection_score']:
            self.best_result['f1_score'] = f1
            self.best_result['epoch_selection_score'] = epoch_selection_score
            self.best_result['threshold'] = th
            self.best_result['epoch'] = epoch
            self.output = evaluate(self.session,self.test_data,self.batch_size,self.predictions ,self.x )
            
    def add_stats(self, epoch,phase,output,labels,loss): 
        print("The "+ phase.upper() +" stats :")
        print('   Loss: {:.4f}'.format(loss))
        
        th,best_epoch_f1, best_epoch_output = scores(self.thresholds,output,labels)

        if phase == 'train':
            self.f1_train[0] = th
            self.f1_train[1] = best_epoch_f1
            print("   Threshold: {:.2f} F1: {:.4f}".format(self.f1_train[0],self.f1_train[1]))
        elif phase == 'val':
            self.f1_val[0] = th 
            self.f1_val[1] = best_epoch_f1 
            print("   Threshold: {:.2f} F1: {:.4f}".format(self.f1_val[0],self.f1_val[1]))
            self.check(epoch,loss)

            
    def best_output(self):
        print('My stats: Thresholds {}:: F1: {:.4f}, epoch: {}'.format(self.best_result['threshold'], self.best_result['f1_score'],self.best_result['epoch']))
        print('Best scor in competition so far: 0.699')
        return self.output,self.best_result['threshold'],self.best_result['f1_score']

In [22]:
def split_on_class(x,y, percentage = 80, random = True):
    
    data_in_classes = [[],[]]
    
    left_split = []
    right_split = []
    
    for idx,val in enumerate(y):                    #separate the data with target 0 from 1
        data_in_classes[int(val)].append([idx,val]) #save them as (idx,val) to save memory
    
    for class_data in data_in_classes:          #split the classes in two and add them to the left and right split 
        total = len(class_data)
        split = int(total*(percentage/100))
        left_split += class_data[:split]
        right_split += class_data[split:]
    
    if random:                                  #shuffle them so we lose the order of the quetions
        np.random.shuffle(right_split)
        np.random.shuffle(left_split)
      
    # make the x_train,y_train from the left_split
    left_split = np.array(left_split)
    y_left = left_split[:,1]
    left_split = left_split[:,0]
    x_left = []
    for idx_x in left_split:
        x_left.append(x[idx_x])
    del left_split
    
    # make the x_test,y_test from the right_split
    right_split = np.array(right_split)
    y_right = right_split[:,1]
    right_split = right_split[:,0]
    x_right = []
    for idx_x in right_split:
        x_right.append(x[idx_x])
    del right_split
    
            
    return x_left,x_right,y_left,y_right

In [23]:
#x_train, x_test, y_train, y_test  = split_on_class(questions_train,y)
x_train, x_val, y_train, y_val  = split_on_class(questions_train, y)
print("The leght of train data      :",len(x_train))
print("The leght of validation data :",len(x_val))
#print("The leght of test data       :",len(x_test))
print("There are",sum(y_train),"labeled positive in the train data")
print("There are",sum(y_val),"labeled positive in the validation data")
#print("There are",sum(y_test),"labeled positive in the test data")

The leght of train data      : 1044897
The leght of validation data : 261225
There are 64648 labeled positive in the train data
There are 16162 labeled positive in the validation data


In [24]:
batches = Batch(questions_train, y, 400)

In [25]:
len(batches)

3266

In [31]:
def weights(x,drop): 
    with tf.variable_scope('lreg') as scope:
        aux = Bidirectional(LSTM(60, return_sequences=True))(x)
        aux = GlobalMaxPool1D()(aux)
        aux = Dense(16, activation="relu")(aux)
        aux = tf.nn.dropout(aux, drop)
        aux = Dense(1, activation="sigmoid")(aux)
        out = tf.reshape(aux, [-1])

    return out

In [2]:
def run():
    global questions_test
    global embeddigs
    global x_train
    global y_train
    global x_test
    global y_test
    global x_val
    global y_val
    global questions_train
    global y


    batch_size = 350
    thresholds = np.arange(0.2, 0.5, 0.02)
    epochs = 5

    x = tf.placeholder(tf.float32,shape=(None, 60, 300), name = 'x')
    y = tf.placeholder(tf.float32,shape=(None, ),name='y')
    drop = tf.placeholder_with_default(1.0, shape=()) #we need variable drop rate so we can stop the drop layer at evaluation
    predictions = weights(x,drop)
    loss = binary_crossentropy(target = y, output = predictions)

    optimizer = tf.train.AdamOptimizer(learning_rate=1e-3)
    optimizer = tf.contrib.estimator.clip_gradients_by_norm(optimizer, clip_norm=5.0)
    optimizer = optimizer.minimize(loss)
    
    init_op = tf.global_variables_initializer()
    #saver = tf.train.Saver()

    with tf.Session() as session:
        session.run(init_op)
        stats = Stats(questions_test,session,thresholds,batch_size,predictions,x)
        
        for e in range(1, epochs + 1):
            print("\n" + "_" * 80)
            print("Epoch nr", e, ":")
            
            phases =  ['train', 'val']
            for phase in phases:
                average_loss = 0
                output = np.array([])
                labels = np.array([])

                if phase == 'train':
                    batches = Batch(x_train, y_train, batch_size)
                elif phase == 'val':
                    batches = Batch(x_val, y_val, batch_size)

                for batch in tqdm(batches):
                    batch, label =  batch
                    batch = embedding_lookup( batch )
                    label = np.array(label)

                    if phase == 'train':
                        _ ,batch_output ,batch_loss = session.run([optimizer, predictions, loss],\
                                                                  {x:batch, y :label, drop:0.4})
                    else:
                        batch_output ,batch_loss = session.run([predictions, loss], \
                                                                  {x:batch, y :label, drop: 1.0})

                    labels = np.concatenate((labels, label), axis=0)
                    output = np.concatenate((output, batch_output), axis=0)
                    average_loss += np.average(batch_loss)
                average_loss = average_loss/len(batches)
                
                stats.add_stats(e, phase, output, labels, average_loss)                    

        return stats.best_output()

In [33]:
first_output, first_th, first_score = run()
second_output, second_th, second_score = run()
if first_score > second_score:
    output = first_output
    th = first_th
    f1_score = first_score
else:
    output = second_output
    th = second_th
    f1_score = second_score


________________________________________________________________________________
Epoch nr 1 :


100%|██████████████████████████████████████████████████████████████| 2986/2986 [32:43<00:00,  1.79it/s]


The TRAIN stats :
   Loss: 0.0561
   Thresholds 0.20:: Acc: 0.9175 | Precision: 0.4139 | Recall: 0.8007 | F1: 0.5457
   Thresholds 0.22:: Acc: 0.9221 | Precision: 0.4295 | Recall: 0.7873 | F1: 0.5558
   Thresholds 0.24:: Acc: 0.9262 | Precision: 0.4445 | Recall: 0.7735 | F1: 0.5645
   Thresholds 0.26:: Acc: 0.9297 | Precision: 0.4587 | Recall: 0.7595 | F1: 0.5720
   Thresholds 0.28:: Acc: 0.9328 | Precision: 0.4726 | Recall: 0.7451 | F1: 0.5784
   Thresholds 0.30:: Acc: 0.9355 | Precision: 0.4856 | Recall: 0.7286 | F1: 0.5828
   Thresholds 0.32:: Acc: 0.9379 | Precision: 0.4987 | Recall: 0.7112 | F1: 0.5863
   Thresholds 0.34:: Acc: 0.9401 | Precision: 0.5118 | Recall: 0.6910 | F1: 0.5880
   Thresholds 0.36:: Acc: 0.9421 | Precision: 0.5249 | Recall: 0.6685 | F1: 0.5881
   Thresholds 0.38:: Acc: 0.9437 | Precision: 0.5380 | Recall: 0.6419 | F1: 0.5854
   Thresholds 0.40:: Acc: 0.9451 | Precision: 0.5515 | Recall: 0.6044 | F1: 0.5767
   Thresholds 0.42:: Acc: 0.9435 | Precision: 0.5725 

100%|████████████████████████████████████████████████████████████████| 747/747 [04:03<00:00,  3.77it/s]


The VAL stats :
   Loss: 0.0388
   Thresholds 0.20:: Acc: 0.9414 | Precision: 0.5165 | Recall: 0.8211 | F1: 0.6341
   Thresholds 0.22:: Acc: 0.9437 | Precision: 0.5293 | Recall: 0.8100 | F1: 0.6402
   Thresholds 0.24:: Acc: 0.9457 | Precision: 0.5415 | Recall: 0.7982 | F1: 0.6453
   Thresholds 0.26:: Acc: 0.9475 | Precision: 0.5531 | Recall: 0.7868 | F1: 0.6496
   Thresholds 0.28:: Acc: 0.9491 | Precision: 0.5651 | Recall: 0.7732 | F1: 0.6529
   Thresholds 0.30:: Acc: 0.9507 | Precision: 0.5768 | Recall: 0.7604 | F1: 0.6560
   Thresholds 0.32:: Acc: 0.9520 | Precision: 0.5887 | Recall: 0.7459 | F1: 0.6580
   Thresholds 0.34:: Acc: 0.9532 | Precision: 0.5997 | Recall: 0.7307 | F1: 0.6587
   Thresholds 0.36:: Acc: 0.9543 | Precision: 0.6122 | Recall: 0.7132 | F1: 0.6589
   Thresholds 0.38:: Acc: 0.9554 | Precision: 0.6265 | Recall: 0.6925 | F1: 0.6578
   Thresholds 0.40:: Acc: 0.9567 | Precision: 0.6442 | Recall: 0.6689 | F1: 0.6563
   Thresholds 0.42:: Acc: 0.9577 | Precision: 0.6644 | 

...

The VAL stats :
   Loss: 0.0417
   Thresholds 0.20:: Acc: 0.9512 | Precision: 0.5787 | Recall: 0.7779 | F1: 0.6637
   Thresholds 0.22:: Acc: 0.9523 | Precision: 0.5872 | Recall: 0.7701 | F1: 0.6663
   Thresholds 0.24:: Acc: 0.9532 | Precision: 0.5949 | Recall: 0.7624 | F1: 0.6683
   Thresholds 0.26:: Acc: 0.9540 | Precision: 0.6023 | Recall: 0.7545 | F1: 0.6698
   Thresholds 0.28:: Acc: 0.9548 | Precision: 0.6100 | Recall: 0.7461 | F1: 0.6712
   Thresholds 0.30:: Acc: 0.9554 | Precision: 0.6169 | Recall: 0.7374 | F1: 0.6718
   Thresholds 0.32:: Acc: 0.9561 | Precision: 0.6244 | Recall: 0.7286 | F1: 0.6725
   Thresholds 0.34:: Acc: 0.9568 | Precision: 0.6324 | Recall: 0.7197 | F1: 0.6732
   Thresholds 0.36:: Acc: 0.9574 | Precision: 0.6410 | Recall: 0.7094 | F1: 0.6735
   Thresholds 0.38:: Acc: 0.9581 | Precision: 0.6510 | Recall: 0.6964 | F1: 0.6729
   Thresholds 0.40:: Acc: 0.9588 | Precision: 0.6623 | Recall: 0.6806 | F1: 0.6713
   Thresholds 0.42:: Acc: 0.9594 | Precision: 0.6770 | 

100%|██████████████████████████████████████████████████████████████| 2986/2986 [30:50<00:00,  1.97it/s]


The TRAIN stats :
   Loss: 0.0378
   Thresholds 0.20:: Acc: 0.9493 | Precision: 0.5556 | Recall: 0.9000 | F1: 0.6871
   Thresholds 0.22:: Acc: 0.9512 | Precision: 0.5671 | Recall: 0.8929 | F1: 0.6936
   Thresholds 0.24:: Acc: 0.9528 | Precision: 0.5772 | Recall: 0.8857 | F1: 0.6989
   Thresholds 0.26:: Acc: 0.9541 | Precision: 0.5863 | Recall: 0.8779 | F1: 0.7031
   Thresholds 0.28:: Acc: 0.9554 | Precision: 0.5958 | Recall: 0.8700 | F1: 0.7073
   Thresholds 0.30:: Acc: 0.9566 | Precision: 0.6046 | Recall: 0.8621 | F1: 0.7107
   Thresholds 0.32:: Acc: 0.9576 | Precision: 0.6129 | Recall: 0.8532 | F1: 0.7134
   Thresholds 0.34:: Acc: 0.9584 | Precision: 0.6204 | Recall: 0.8423 | F1: 0.7145
   Thresholds 0.36:: Acc: 0.9591 | Precision: 0.6284 | Recall: 0.8310 | F1: 0.7156
   Thresholds 0.38:: Acc: 0.9598 | Precision: 0.6362 | Recall: 0.8177 | F1: 0.7156
   Thresholds 0.40:: Acc: 0.9604 | Precision: 0.6444 | Recall: 0.8024 | F1: 0.7148
   Thresholds 0.42:: Acc: 0.9610 | Precision: 0.6714 

100%|████████████████████████████████████████████████████████████████| 747/747 [03:40<00:00,  4.05it/s]


The VAL stats :
   Loss: 0.0478
   Thresholds 0.20:: Acc: 0.9538 | Precision: 0.6016 | Recall: 0.7483 | F1: 0.6670
   Thresholds 0.22:: Acc: 0.9544 | Precision: 0.6077 | Recall: 0.7424 | F1: 0.6683
   Thresholds 0.24:: Acc: 0.9548 | Precision: 0.6119 | Recall: 0.7353 | F1: 0.6679
   Thresholds 0.26:: Acc: 0.9552 | Precision: 0.6169 | Recall: 0.7291 | F1: 0.6683
   Thresholds 0.28:: Acc: 0.9556 | Precision: 0.6213 | Recall: 0.7220 | F1: 0.6679
   Thresholds 0.30:: Acc: 0.9560 | Precision: 0.6261 | Recall: 0.7156 | F1: 0.6679
   Thresholds 0.32:: Acc: 0.9564 | Precision: 0.6315 | Recall: 0.7086 | F1: 0.6679
   Thresholds 0.34:: Acc: 0.9568 | Precision: 0.6373 | Recall: 0.7010 | F1: 0.6676
   Thresholds 0.36:: Acc: 0.9573 | Precision: 0.6437 | Recall: 0.6939 | F1: 0.6679
   Thresholds 0.38:: Acc: 0.9578 | Precision: 0.6505 | Recall: 0.6859 | F1: 0.6678
   Thresholds 0.40:: Acc: 0.9582 | Precision: 0.6570 | Recall: 0.6776 | F1: 0.6671
   Thresholds 0.42:: Acc: 0.9586 | Precision: 0.6648 | 

In [ ]:
print(output)
print(th)
print(f1_score)

In [ ]:
out = output
output[output<=th] = 0
output[output>th] = 1
out = output.astype(int)